## Libraries Used

In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

## Load Data

Here we load in the local data from the .json and .txt files then parse that data to collect the desired features in the form of number of words per headline, anger words and the associated intensity score. All of the data is then stored into pandas dataframes for ease of use in the ML models.

In [96]:
def parseJson(fname):
    for line in open(fname, 'r'):
        yield eval(line)
def run():
    data = list(parseJson('./Sarcasm_Headlines_Dataset.json'))
    emo = pd.read_csv('./NRC-Emotion-Intensity-Lexicon-v1.txt', delimiter = "\t")
    global df
    df = pd.DataFrame.from_dict(data)
    df['numWords'] = df['headline'].str.split().str.len() # Parses individual headlines into the total individual words within

    df['emotion'] = 0
    df['intensity'] = 0
    angerList = [] #Creating shorter lists parsed from the Lexicon to speed up runtime.
    intensityList = {}
    for i, row in emo.iterrows():
        if emo['emotion'][i] == "anger":
            angerList.append(emo['word'][i])
            intensityList[emo['word'][i]] = emo['emotion-intensity-score'][i]

    s = set(df['headline'])#Converted to set to speed up the process
    a = set(angerList)
    for i,x in enumerate(df['headline']):#Simple loop to match all indivdual words within all headlines with words displaying "anger", while also grabbing intensity.
        res = x.split()
        for j in res:
            if j in a:
                df.at[i,'intensity'] += intensityList[j]
                df.at[i,'emotion']+=1
run()

## Naive Bayes

Here we run the first Naive Bayes model were we can pass in a desired value for the ngram range on the data. This is then fed into the model to achieve scores.

In [98]:
def naiveBayes(x,y):
    #Set the CountVectorizer with the desired n-grams
    cv = CountVectorizer(ngram_range=(x,y), min_df = 1)
    #Use the configred cv to transform the dataframe
    features = cv.fit_transform(df['headline'])
    
    #features = np.array(headlines.todense())
    #print(len(cv.get_feature_names_out()))
    #print(cv.get_feature_names_out()[:20])
    #print(cv.get_params())
    
    #Create the x, y values containing the data matrix and sarcasm value matrix then pass them into the model
    X = features
    y = df['is_sarcastic'] 
    clf = MultinomialNB()

    #Calculate the Acc and F1 scores useing 10-fold Cross-Validation
    resultAcc = cross_val_score(clf, X, y, cv = 10, scoring='accuracy')
    resultF1 = cross_val_score(clf, X, y, cv = 10, scoring='f1')
    print('Cross-Val-Acc:', resultAcc)
    print('Average-Acc:',sum(resultAcc)/len(resultAcc))
    print('Cross-Val-F1:', resultF1)
    print('Average-F1:',sum(resultF1)/len(resultF1))
naiveBayes(1,3)

Cross-Val-Acc: [0.84730957 0.85953878 0.85814116 0.85639413 0.86827393 0.86722572
 0.87176799 0.86477987 0.85814116 0.86158686]
Average-Acc: 0.8613159184786074
Cross-Val-F1: [0.83712262 0.85077951 0.84918276 0.84760845 0.85874859 0.85946746
 0.86372076 0.85511045 0.84656085 0.85168539]
Average-F1: 0.8519986844998219


## Naive Bayes #2

Here we run the first Naive Bayes model were we can pass in a desired value for the ngram range on the data. This is then fed into the model to achieve scores with the additional features being combined into a dataframe beforehand.

In [87]:
def naiveBayes(x,y):
    #Set the CountVectorizer with the desired n-grams
    cv = CountVectorizer(ngram_range=(x,y), min_df = 1)
    #Form the dataframe with additional features
    df['naiveBayes'] = df['headline']+" "+df['numWords'].astype(str)+" "+df['emotion'].astype(str)+" "+df['intensity'].astype(str)
    #Use the configred cv to transform the dataframe
    features = cv.fit_transform(df['naiveBayes'])
    
    ##features = np.array(headlines.todense())
    #print(len(cv.get_feature_names_out()))
    #print(cv.get_feature_names_out()[:20])

    #Create the x, y values containing the data matrix and sarcasm value matrix then pass them into the model
    X = features
    y = df['is_sarcastic'] 
    clf = MultinomialNB()
    #Calculate the Acc and F1 scores useing 10-fold Cross-Validation
    resultAcc = cross_val_score(clf, X, y, cv = 10, scoring='accuracy')
    resultF1 = cross_val_score(clf, X, y, cv = 10, scoring='f1')
    print('Cross-Val-Acc:', resultAcc)
    print('Average-Acc:',sum(resultAcc)/len(resultAcc))
    print('Cross-Val-F1:', resultF1)
    print('Average-F1:',sum(resultF1)/len(resultF1))
naiveBayes(1,3)

Cross-Val-Acc: [0.84591195 0.86582809 0.85709294 0.85499651 0.87386443 0.86792453
 0.87281621 0.86477987 0.85709294 0.86263544]
Average-Acc: 0.8622942919441703
Cross-Val-F1: [0.8348933  0.85671642 0.8475587  0.84612532 0.86464192 0.85989622
 0.86428039 0.85543519 0.8458349  0.85231116]
Average-F1: 0.8527693524375719


## Logistic Regression

Here we run the first Logistic Regression model were we can pass in a desired value for the ngram range on the data. This is then fed into the model to achieve scores.

In [51]:
def logisticRegression(x,y):
    #Set the CountVectorizer with the desired n-grams
    cv = CountVectorizer(ngram_range=(x,y), min_df = 1)
    #Use the configred cv to transform the dataframe
    features = cv.fit_transform(df['headline'])
    
    #features = np.array(headlines.todense())
    #print(len(cv.get_feature_names_out()))
    
    #Create the x, y values containing the data matrix and sarcasm value matrix then pass them into the model
    X = features
    y = df['is_sarcastic']
    clf = LogisticRegression(max_iter=1000)#verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

    #Calculate the Acc and F1 scores useing 10-fold Cross-Validation
    resultAcc = cross_val_score(clf, X, y, cv = 10, scoring='accuracy')
    resultF1 = cross_val_score(clf, X, y, cv = 10, scoring='f1')
    print('Cross-Val-Acc:', resultAcc)
    print('Average-Acc:',sum(resultAcc)/len(resultAcc))
    print('Cross-Val-F1:', resultF1)
    print('Average-F1:',sum(resultF1)/len(resultF1))
logisticRegression(1,3)

Cross-Val-Acc: [0.84940601 0.84451433 0.85429769 0.84696017 0.86512928 0.86408106
 0.86338225 0.8602376  0.85429769 0.85145054]
Average-Acc: 0.8553756621433182
Cross-Val-F1: [0.84229784 0.83753195 0.84841876 0.84026258 0.85902118 0.85756133
 0.85703839 0.85196151 0.84595493 0.84346225]
Average-F1: 0.848351072188233


## Logistic Regression #2

Here we run the first Logistic Regression model were we can pass in a desired value for the ngram range on the data. This is then fed into the model to achieve scores with the additional features being combined into a dataframe beforehand.

In [89]:
def logisticRegression(x,y):
    #Set the CountVectorizer with the desired n-grams
    cv = CountVectorizer(ngram_range=(x,y), min_df = 1)
    #Form the dataframe with additional features
    df['logisticRegression'] = df['headline']+" "+df['numWords'].astype(str)+" "+df['emotion'].astype(str)+" "+df['emotion'].astype(str)
    #Use the configred cv to transform the dataframe
    features = cv.fit_transform(df['logisticRegression'])
    
    #features = np.array(headlines.todense())
    #print(len(cv.get_feature_names_out()))
    
    #Create the x, y values containing the data matrix and sarcasm value matrix then pass them into the model
    X = features
    y = df['is_sarcastic']
    clf = LogisticRegression(max_iter=1000)#verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
    clf.fit(X, y)
    #Calculate the Acc and F1 scores useing 10-fold Cross-Validation
    resultAcc = cross_val_score(clf, X, y, cv = 10, scoring='accuracy')
    resultF1 = cross_val_score(clf, X, y, cv = 10, scoring='f1')
    print('Cross-Val-Acc:', resultAcc)
    print('Average-Acc:',sum(resultAcc)/len(resultAcc))
    print('Cross-Val-F1:', resultF1)
    print('Average-F1:',sum(resultF1)/len(resultF1))
    for i in range(len(df['naiveBayes'])):
        pre = clf.predict(cv.transform([df['naiveBayes'][i]]))
        if pre != df['is_sarcastic'][i]:
            print('pos', i)
            print('Actual: ',df['is_sarcastic'][i])
            print('Predict: ',pre, df['naiveBayes'][i])
logisticRegression(1,3)

Cross-Val-Acc: [0.85185185 0.84975542 0.8602376  0.85115304 0.86792453 0.86862334
 0.86512928 0.86582809 0.860587   0.85564488]
Average-Acc: 0.8596735026163316
Cross-Val-F1: [0.84491587 0.84283626 0.85358712 0.84475219 0.8622449  0.86237189
 0.85912409 0.85840708 0.85314685 0.84788214]
Average-F1: 0.8529268377091501
pos 656
Actual:  0
Predict:  [1] celebrate older americans month by fighting senior poverty 8 2 1.18
pos 863
Actual:  0
Predict:  [1] a different kind of mom 5 0 0.0
pos 14388
Actual:  1
Predict:  [0] your dog died 3 0 0.0
pos 21030
Actual:  0
Predict:  [1] left behind 2 0 0.0
pos 22811
Actual:  0
Predict:  [1] stepping back into now 4 0 0.0
pos 23882
Actual:  0
Predict:  [1] breaking up with god 4 0 0.0
pos 25776
Actual:  0
Predict:  [1] breaking uniform 2 0 0.0


# Written Component

### 1. Present and discuss the results of the two clasifciation models and different feature sets.

<center>

|       Model      |Acc-Score    |F1-Score  |
| :--              | :----:     |   :----:  |
|   NB N-Grams     |   0.8613         |    0.8519       |
|   NB Features    |   0.8622         |    0.8527       |
|   LR N-Grams     |   0.8553         |    0.8483       |
|   LR Features    |   0.8596         |    0.8529       |

Figure #1
</center>
Shown above in fig #1 are the averaged results of F1 and Acc score testing using 10-fold cross-validation on all four of my designed models with differing feature sets. Of note are that these models performed differently in regards to the f1 & acc score with there being no "clear" winner. This showed that the Logistic regression model with the additional features beyond N-grams was capable of identifying false negatives and false positives, whereas the Naive Bayes model was more attuned to True positives and True negatives. Of note is that while the results are of similar scores the performance behind such algorithms differed dramatically, average performance for both logistic regression algorithms was found to be 1m:39.9s whereas Naive Bayes was 1.9s, showing it's dramatically increased performance on the dataset.

### 2. Describe your best model and a justification of the features that you used or did not use. Why do you think they are helpful or not? You must explicitly list the features you tried here.




For this series of four models the best F1 results were found in my Logistic Regression model with additional features. The average F1 score in my trials for this model were found to be 0.8529, as seen above in the previously posted fig #1, while the greatest Acc-Score was seen in the Naive Bayes model with additional features. Both of these models with the features in fig #2 performed the best across my trials.

&nbsp;
<center>

|       Model      |Features    |
| :--              | :----:     |
|   NB N-Grams     |   N-Grams        |
|   NB Features    |   N-Grams, Num of words, Number of Angry words, Anger-Intensity         |
|   LR N-Grams     |   N-Grams        |
|   LR Features    |   N-Grams, Num of words, Number of Angry words, Anger-Intensity        |

Figure #2
</center>
Seen above in fig #2 I made use of a collection of features that I felt would best show a correlation to the possibility of sarcasm, while also attempint to avoid misidentification on other headlines. To start with, my reasoning as to why I made use of words depicting anger as opposed to sadness or trust due to the likelyhood of such an emotion being used sarcasticly, often in a mocking manner as opposed to other emotions were the correlation might not be as strongly seen. The same reasoning followed for the use of the intensity of said language, with my reasoning being that higher intensity of anger might lead to an increased likelyhood of sarcastic comments appearing in a headline. lastly, I added the feature containing the number of words due to the thought process that short and concise headlines might trend less sarcastic, whereas ones that trial on could increase that likelyhood.

### 3. Conduct an error analysis. For your best perofrming model, select 3 interesting examples that did not work. Indicate why you think they were classified incorrectly.

For instances where errors in classification occured, three interesting examples that I found were "celebrate older americans month by fighting senior poverty", "a different kind of mom ", and "Your dog died ". For the headline "celebrate older americans month by fighting senior poverty" it was misclassified as being sarcastic, likely due to the use of two negative words with an intensity of 1.18 along with vocabulary and form similar to that found in other sarcastic headlines. Next, for the headline "a different kind of mom" despite not having negative words it was still predicted as being saracstic, which I think is because of the more casual nature of the headline conforming to a vocabulary that could be considered sarcastic depending on the audience. And lastly for the headline "your dog died" which was predicted as not being sarcastic wherein it actually was I believe this was due to its short and matter of fact nature, which contrasts to the average sarcastic headline that might use upwards of a dozen words.

### 4. If you could add additional features, what features do you think would help improve performance?
    
For this model since we're dealing with sarcasm I think having additional features relating to the background or specifications of the source would be benficial. So for example, having a feature that keeps track of the likelyhood of a source providing a sarcastic comment i.e. "The Onion" might help prevent the model from misidentifying sarcasm from other sources since nearly all headlines from "The onion" are sarcastic. Also features identifying the category of news, such as podcast, scientifc journal, paper, or others sources might increase percision due to differing mehtods in jounralism seen accross mediums. These features would help give additional background context missing from just observing the resulting headlines.